In [1]:
'''
#testing
import torch
import time
def gputester(MATRIX_SIZE):
    # Define the size of the matrices (adjust for your system)

    # Create random matrices
    cpu_tensor1 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)
    cpu_tensor2 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)

    # Move tensors to GPU (if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gpu_tensor1 = cpu_tensor1.to(device)
    gpu_tensor2 = cpu_tensor2.to(device)

    ### **🔹 CPU Benchmark**
    start_cpu = time.time()
    cpu_result = torch.matmul(cpu_tensor1, cpu_tensor2)  # Matrix multiplication on CPU
    end_cpu = time.time()
    cpu_time = end_cpu - start_cpu
    print(f"✅ CPU Time: {cpu_time:.4f} seconds")

    ### **🔹 GPU Benchmark**
    # Ensure the GPU is warmed up
    torch.matmul(gpu_tensor1, gpu_tensor2)

    # Measure GPU performance
    torch.cuda.synchronize()  # Ensure operations are completed before timing
    start_gpu = time.time()
    gpu_result = torch.matmul(gpu_tensor1, gpu_tensor2)  # Matrix multiplication on GPU
    torch.cuda.synchronize()  # Ensure GPU computation is finished before stopping time
    end_gpu = time.time()
    gpu_time = end_gpu - start_gpu
    print(f"🚀 GPU Time: {gpu_time:.4f} seconds")

    ### **🔹 Speedup Factor**
    if torch.cuda.is_available():
        speedup = cpu_time / gpu_time
        print(f"🔥 GPU is {speedup:.2f}x faster than CPU!")
    else:
        print("⚠️ CUDA is not available! Running only on CPU.")

gputester(10000)
'''

'\n#testing\nimport torch\nimport time\ndef gputester(MATRIX_SIZE):\n    # Define the size of the matrices (adjust for your system)\n\n    # Create random matrices\n    cpu_tensor1 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)\n    cpu_tensor2 = torch.randn(MATRIX_SIZE, MATRIX_SIZE)\n\n    # Move tensors to GPU (if available)\n    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")\n    gpu_tensor1 = cpu_tensor1.to(device)\n    gpu_tensor2 = cpu_tensor2.to(device)\n\n    ### **🔹 CPU Benchmark**\n    start_cpu = time.time()\n    cpu_result = torch.matmul(cpu_tensor1, cpu_tensor2)  # Matrix multiplication on CPU\n    end_cpu = time.time()\n    cpu_time = end_cpu - start_cpu\n    print(f"✅ CPU Time: {cpu_time:.4f} seconds")\n\n    ### **🔹 GPU Benchmark**\n    # Ensure the GPU is warmed up\n    torch.matmul(gpu_tensor1, gpu_tensor2)\n\n    # Measure GPU performance\n    torch.cuda.synchronize()  # Ensure operations are completed before timing\n    start_gpu = time.time()\n    g

In [107]:
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.card_utils import gen_cards, estimate_hole_card_win_rate

class RuleBasedPokerBot(BasePokerPlayer):

    def declare_action(self, valid_actions, hole_card, round_state):
        """ Selects Fold, Call, or Raise based on hand strength. """
        
        # Convert cards into the correct format
        hand_strength = estimate_hole_card_win_rate(
            nb_simulation=1000, 
            nb_player=3, 
            hole_card=gen_cards(hole_card),  # Convert to card objects
            community_card=gen_cards(round_state["community_card"])  # Convert community cards
        )

        action, amount = "fold", 0  

        if hand_strength >= 0:
            raise_action = next((a for a in valid_actions if a["action"] == "raise"), None)
            if raise_action:
                action, amount = "raise", raise_action["amount"]["min"]
            else:
                action, amount = "call", 0  # If raising is unavailable, call

        elif "call" in [a["action"] for a in valid_actions]:  
            action, amount = "call", 0  # Call doesn't need a bet amount
        return action, amount  # Must return (action, amount)

    def receive_game_start_message(self, game_info): pass  
    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"RuleBasedPokerBot Hole Cards: {hole_card}\n")
    def receive_street_start_message(self, street, round_state): pass
    def receive_game_update_message(self, action, round_state): pass  
    def receive_round_result_message(self, winners, hand_info, round_state): pass



class AlwaysCallBot(BasePokerPlayer):
    def declare_action(self, valid_actions, hole_card, round_state):
        """ Always calls when possible, otherwise folds. """
        action, amount = "fold", 0  # Default to folding
        # Check if calling is an option
        if any(a["action"] == "call" for a in valid_actions):
            call_action_info = valid_actions[1]
            action, amount = "call", call_action_info["amount"]  # Call doesn't require a bet amount
        
        return action, amount

    def receive_game_start_message(self, game_info): pass  
    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"AlwaysCallBot Hole Cards: {hole_card}\n")              
    def receive_street_start_message(self, street, round_state): pass
    def receive_game_update_message(self, action, round_state): pass  
    def receive_round_result_message(self, winners, hand_info, round_state): pass


In [109]:
from pypokerengine.api.game import setup_config, start_poker

config = setup_config(max_round=1, initial_stack=100, small_blind_amount=5)
config.register_player(name="RuleBasedPokerBot", algorithm=RuleBasedPokerBot())
config.register_player(name="AlwaysCallBot", algorithm=AlwaysCallBot())

game_result = start_poker(config, verbose=1)


RuleBasedPokerBot Hole Cards: ['C9', 'DA']

AlwaysCallBot Hole Cards: ['H5', 'S8']

Started the round 1
Street "preflop" started. (community card = [])
"AlwaysCallBot" declared "call:10"
"RuleBasedPokerBot" declared "raise:15"
"AlwaysCallBot" declared "call:15"
Street "flop" started. (community card = ['D2', 'C2', 'CJ'])
"AlwaysCallBot" declared "call:0"
"RuleBasedPokerBot" declared "raise:10"
"AlwaysCallBot" declared "call:10"
Street "turn" started. (community card = ['D2', 'C2', 'CJ', 'D7'])
"AlwaysCallBot" declared "call:0"
"RuleBasedPokerBot" declared "raise:10"
"AlwaysCallBot" declared "call:10"
Street "river" started. (community card = ['D2', 'C2', 'CJ', 'D7', 'D5'])
"AlwaysCallBot" declared "call:0"
"RuleBasedPokerBot" declared "raise:10"
"AlwaysCallBot" declared "call:10"
"['AlwaysCallBot']" won the round 1 (stack = {'RuleBasedPokerBot': 55, 'AlwaysCallBot': 145})
